In [3]:
import pandas as pd

In [4]:
pd.__version__

'1.4.2'

In [5]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [4]:
!pip install pyarrow

     |████████████████████████████████| 42.3 MB 27.1 MB/s eta 0:00:01


In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [7]:
import sklearn

In [9]:
sklearn.__version__

'1.0.2'

In [8]:
num_columns = len(df.columns)

In [9]:
print("Number of columns:", num_columns)

Number of columns: 19


In [10]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [11]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [12]:
std_duration = df['duration'].std()

In [13]:
print("Standard deviation of trip duration (in minutes):", std_duration)

Standard deviation of trip duration (in minutes): 34.851053592192876


In [14]:
total_before = len(df)

In [15]:
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

In [16]:
total_after = len(df_filtered)

In [17]:
fraction_remaining = total_after / total_before

In [18]:
print("Fraction of records remaining after removing outliers:", fraction_remaining)

Fraction of records remaining after removing outliers: 0.9778326020432945


In [19]:
df_filtered['PULocationID'] = df_filtered['PULocationID'].astype(str)


/tmp/ipykernel_8976/1787443611.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['PULocationID'] = df_filtered['PULocationID'].astype(str)


In [21]:
df_filtered = df_filtered.copy() 
df_filtered['PULocationID'] = df_filtered['PULocationID'].astype(str)
df_filtered['DOLocationID'] = df_filtered['DOLocationID'].astype(str)

In [22]:
dicts = df_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [23]:
print(dicts[:5])

[{'PULocationID': '186', 'DOLocationID': '79'}, {'PULocationID': '140', 'DOLocationID': '236'}, {'PULocationID': '236', 'DOLocationID': '79'}, {'PULocationID': '79', 'DOLocationID': '211'}, {'PULocationID': '211', 'DOLocationID': '148'}]


In [24]:
dv = DictVectorizer()
X = dv.fit_transform(dicts)

NameError: name 'DictVectorizer' is not defined

In [25]:
from sklearn.feature_extraction import DictVectorizer

In [26]:
dv = DictVectorizer()
X = dv.fit_transform(dicts)

In [27]:
print("Feature matrix shape:", X.shape)

Feature matrix shape: (2898906, 518)


In [28]:
print("Number of features (columns):", X.shape[1])

Number of features (columns): 518


In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [30]:
y = df_filtered['duration'].values

In [31]:
lr = LinearRegression()
lr.fit(X, y)

LinearRegression()

In [32]:
y_pred = lr.predict(X)

In [33]:
rmse = np.sqrt(mean_squared_error(y, y_pred))

In [34]:
print("RMSE on training data:", rmse)

RMSE on training data: 7.946173359625103


In [36]:
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')


In [37]:
df_val['tpep_pickup_datetime'] = pd.to_datetime(df_val['tpep_pickup_datetime'])
df_val['tpep_dropoff_datetime'] = pd.to_datetime(df_val['tpep_dropoff_datetime'])

In [38]:
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60

In [39]:
df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)].copy()

In [40]:
df_val['PULocationID'] = df_val['PULocationID'].astype(str)
df_val['DOLocationID'] = df_val['DOLocationID'].astype(str)

In [41]:
val_dicts = df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [42]:
X_val = dv.transform(val_dicts)  # <- NOTE: use transform, NOT fit_transform

In [43]:
y_val = df_val['duration'].values

In [44]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [45]:
y_pred_val = lr.predict(X_val)

In [46]:
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print("Validation RMSE (February 2024):", rmse_val)

Validation RMSE (February 2024): 7.816056786135075
